In [1]:
!pip install transformers torch nltk --quiet
import nltk
nltk.download('vader_lexicon')

# ---------------- SENTIMENT LOGIC ---------------- #
from transformers import pipeline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

try:
    hf_model = pipeline("sentiment-analysis")
except:
    hf_model = None

vader = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    text = text.strip()
    if not text:
        return {"label": "NEUTRAL", "score": 0.0}

    # Try HF model first
    if hf_model:
        res = hf_model(text[:512])[0]
        return {
            "label": res["label"].upper(),
            "score": float(res["score"])
        }

    # Otherwise fallback Vader
    score = vader.polarity_scores(text)["compound"]
    if score >= 0.05:
        return {"label": "POSITIVE", "score": score}
    elif score <= -0.05:
        return {"label": "NEGATIVE", "score": abs(score)}
    else:
        return {"label": "NEUTRAL", "score": abs(score)}


# ---------------- CHATBOT SYSTEM ---------------- #
class SentimentChatbot:
    def __init__(self):
        self.history = []  # stores (text, sentiment)

    def respond_to(self, user_msg):
        sentiment = analyze_sentiment(user_msg)
        self.history.append(sentiment)

        # Simple replies based on sentiment
        if sentiment['label'] == "NEGATIVE":
            return "I'm sorry to hear that. Can you tell me more?", sentiment
        elif sentiment['label'] == "POSITIVE":
            return "That's great to hear! What else would you like to share?", sentiment
        else:
            return "I get it. Please continue.", sentiment

    def overall_sentiment(self):
        pos = sum(1 for s in self.history if s['label']=="POSITIVE")
        neg = sum(1 for s in self.history if s['label']=="NEGATIVE")
        neu = sum(1 for s in self.history if s['label']=="NEUTRAL")

        total = len(self.history)
        if total == 0:
            return "NEUTRAL", 0.0

        if pos > neg:
            final = "POSITIVE"
        elif neg > pos:
            final = "NEGATIVE"
        else:
            final = "NEUTRAL"

        score = round((pos - neg) / total, 3)
        return final, score


# ---------------- RUN INTERACTIVE CHATBOT ---------------- #
bot = SentimentChatbot()

print("🤖 Chatbot started! Type 'exit' to finish.\n")

while True:
    user = input("You: ")
    if user.lower() == "exit":
        final_label, final_score = bot.overall_sentiment()
        print("\n📌 Final Overall Sentiment:", final_label, "| Score:", final_score)
        print("Goodbye! 👋")
        break

    reply, senti = bot.respond_to(user)
    print(f"Bot: {reply}")
    print(f"🔍 Sentiment => {senti['label']} ({senti['score']})\n")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


🤖 Chatbot started! Type 'exit' to finish.

You: hello
Bot: That's great to hear! What else would you like to share?
🔍 Sentiment => POSITIVE (0.9995185136795044)

You: world
Bot: That's great to hear! What else would you like to share?
🔍 Sentiment => POSITIVE (0.9980717301368713)

You: Hi, I am feeling good today
Bot: That's great to hear! What else would you like to share?
🔍 Sentiment => POSITIVE (0.9998602867126465)

You: But sometimes I feel ignored
Bot: I'm sorry to hear that. Can you tell me more?
🔍 Sentiment => NEGATIVE (0.9946010112762451)

You: 
Bot: I get it. Please continue.
🔍 Sentiment => NEUTRAL (0.0)

You: exit

📌 Final Overall Sentiment: POSITIVE | Score: 0.4
Goodbye! 👋
